# Importing stuff
---


In [1]:
import aacgmv2, time
import pandas as pd
import numpy as np
from multiprocessing import Pool
import math, os, shutil
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from scipy.io import readsav
import pymap3d as pm
import glob
import datetime, statistics
from aetherpy.io import read_routines
from math import cos, radians, sin, sqrt
from scipy import spatial, signal

from spacepy.coordinates import Coords
from spacepy.time import Ticktock
import fnmatch


from mpl_toolkits.basemap import Basemap
from importlib import reload
import xarray as xr
import geopandas
import sys

from scipy.interpolate import LinearNDInterpolator, interp1d

%matplotlib inline

# Settings
---


In [2]:


dtime_storm_start = datetime.datetime(2011,5,21,13,40) 
plot_start_delta  = 4   #hours before storm onset to start making plots
plot_end_delta    = 8  # hours after storm onset to end plots. Set to -1 to run for the whole time


gitm_cols = ['Rho', '[O(!U3!NP)]', '[O!D2!N]', '[N!D2!N]', '[N(!U4!NS)]', '[NO]', '[He]', '[N(!U2!ND)]', '[N(!U2!NP)]', 
             '[H]', '[CO!D2!N]', '[O(!U1!ND)]', 'Temperature', 'V!Dn!N(east)', 'V!Dn!N(north)', 'V!Dn!N(up)', 'V!Dn!N(up,O(!U3!NP))',
             'V!Dn!N(up,O!D2!N)', 'V!Dn!N(up,N!D2!N)', 'V!Dn!N(up,N(!U4!NS))', 'V!Dn!N(up,NO)', 'V!Dn!N(up,He)', '[O_4SP_!U+!N]', 
             '[NO!U+!N]', '[O!D2!U+!N]', '[N!D2!U+!N]', '[N!U+!N]', '[O(!U2!ND)!U+!N]', '[O(!U2!NP)!U+!N]', '[H!U+!N]', '[He!U+!N]', 
             '[e-]', 'eTemperature', 'iTemperature', 'V!Di!N(east)', 'V!Di!N(north)', 'V!Di!N(up)'] # which gitm columns do you want plotted?
#TODO: test with not all columns!

gitm_path = "/petastore/phil/GITM/SimStormForSal/GITM-outputs/"


gitm_alt_idxs = -1 #set this to -1 if you want all altitudes
gitm_keo_lons = [-90,2,90,-178]

global_lat_lim = None # will limit all plots latitude. Must be none or less than keo_lat_lim
# ^^ Needs to be tested.

keo_lat_lim = 65 # limits keos to +/- degrees of lat. 

OVERWRITE = True # be careful!

num_pool_workers = 40 # number of workers to use in multithreading jobs. Set to 1 if you don't know what this means.

both_map_plots = True # make both_map_plots filtered and raw maps? if you only want one put it as a str ('raw'/'filt')

sample_rate_min = 5 #min
low_cut = 100 # min, lowest freq wave the filter will allow thru
high_cut = 30 # min, highest freq the filter will allow thru

diff_vs = [1,2,3,5,10,20]



In [3]:
gitm_keo_save_path = "/home/sxe180002/scratch/made_plots/SimStormPaper/keos/"
gitm_map_save_path = "/home/sxe180002/scratch/made_plots/SimStormPaper/maps/"

# Do GITM Plots First
---

In [4]:
gitm_files = np.sort(glob.glob(gitm_path+'3DALL*'))

gitm_dtimes = []
for i in gitm_files:
    yy, MM, dd, hr, mm, sec = i[-17:-15], i[-15:-13], i[-13:-11], i[-10:-8], i[-8:-6], i[-6:-4]
    gitm_dtimes.append(datetime.datetime(int('20' + yy), int(MM), int(dd), int(hr), int(mm), int(sec)))
    
storm_start_index = np.argmin(np.abs(np.array(gitm_dtimes)-dtime_storm_start))
plot_start_idx = np.argmin(np.abs(np.array(gitm_dtimes)-(dtime_storm_start - datetime.timedelta(hours = plot_start_delta))))
plot_end_idx = np.argmin(np.abs(np.array(gitm_dtimes)-(dtime_storm_start + datetime.timedelta(hours = plot_end_delta)))) if plot_end_delta != -1 else -1

# #OVERRIDE varnames
# plot_start_idx = storm_start_index - 2
# plot_end_idx = storm_start_index + 15

## Read in all gitm files

In [5]:
# Importing GITM read routines
from utility_programs.read_routines import GITM

def make_filter(params = None):
    # Define the cutoff frequencies
    lowcut = 1/(100/60)  # 100 minutes in units of sample^-1
    highcut = 1/(30/60) # 30 minutes in units of sample^-1

    # Define the Butterworth filter
    nyquist = 0.5 * 5 # 5 minutes is the sampling frequency
    low = lowcut / nyquist
    high = highcut / nyquist
    sos = signal.butter(2, [low, high], btype='bandstop', output='sos')
    return sos

def make_fits(gitm_bins):
    """
    calculate bandpass filter for all data previously read in.
    
    inputs: nparray of gitmdata
    
    returns:
    fits: np array indexed at fits[time][col][ilon][ilat][ialt]

    
    todo: you can thread this by splitting the alts into different threads.
    then just append the fits_full later.
    
    """
    sos = make_filter()

    filtered_arr = signal.sosfiltfilt(sos, gitm_bins, axis=0)
    return filtered_arr

In [6]:
times, gitm_grid, gitm_bins, gitmvars = GITM.read_gitm_into_nparrays(gitm_path, dtime_storm_start, t_start_idx=4, t_end_idx=8)
lats, lons, alts  = np.unique(gitm_grid['latitude']), np.unique(gitm_grid['longitude']), np.unique(gitm_grid['altitude'])

## Now to convert into xarray


In [7]:
# CONVERSION GOES LIKE THIS:
#---------------------------
#gitm_bins(array) -> datadict(dict) -> ds(xr.Dataset)

# Use this gitmvars when converting to xarray. 
# xarray doesn't like "!" and other special characters
xrgitmvars = ['Rho', 'O_3p_', 'O_2', 'N_2', 'N_4s_', 'NO', 'He', 'N_2d_', 'N_2p_', 'H', 'CO_2', 'O_1d_', 'Temperature', 'v_n_east', 'v_n_north', 'v_n_up', 'v_n_up_O_3p_', 'v_n_up_2n_', 'v_n_up_N_2', 'v_n_up_N_4s_', 'v_n_up_NO', 'v_n_up_He', 'O_4_spPlus_', 'NOPlus', 'O_2Plus', 'N_2Plus', 'NPlus', 'O_2dPlus', 'O_2pPlus', 'HPlus', 'HePlus', 'eMinus', 'eTemperature', 'iTemperature', 'v_i_east', 'v_i_north', 'v_i_up']
dimnames = ['time', 'lon', 'lat', 'alt']

# Loop through data for conversion to dict
datadict={}
for n, name in enumerate(xrgitmvars):
    datadict[name] = (dimnames, gitm_bins[:,n,:,:,:])
lats, lons, alts  = np.unique(gitm_grid['latitude']), np.unique(gitm_grid['longitude']), np.unique(gitm_grid['altitude'])

# Convert to xarray
ds = xr.Dataset(data_vars=datadict,\
                coords=dict(\
                time=times,\
                lon=lons,\
                lat=lats,\
                alt=alts)
)
del datadict

In [11]:
# And now save and compress...
encoding = {}
for n, name in enumerate (xrgitmvars):
    encoding[name] = ({'zlib': True, 'complevel': 9})
ds.encoding=encoding
ds.to_netcdf('/petastore/phil/GITM/SimStormForSal/example.nc', encoding=encoding)

In [9]:
ds

<xarray.Dataset>
Dimensions:       (time: 144, lon: 90, lat: 180, alt: 50)
Coordinates:
  * time          (time) datetime64[ns] 2011-05-21T09:40:00 ... 2011-05-21T21...
  * lon           (lon) float64 -178.0 -174.0 -170.0 ... 170.0 174.0 178.0
  * lat           (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * alt           (alt) float64 1e+05 1.017e+05 1.035e+05 ... 7.29e+05 7.515e+05
Data variables: (12/37)
    Rho           (time, lon, lat, alt) float64 5.686e-07 ... 5.615e-14
    O_3p_         (time, lon, lat, alt) float64 5.625e+17 5.26e+17 ... 1.82e+12
    O_2           (time, lon, lat, alt) float64 2.087e+18 ... 2.275e+09
    N_2           (time, lon, lat, alt) float64 9.434e+18 ... 1.471e+11
    N_4s_         (time, lon, lat, alt) float64 1.903e+11 2.856e+11 ... 9.86e+09
    NO            (time, lon, lat, alt) float64 2.492e+12 2.59e+12 ... 1.385e+07
    ...            ...
    eMinus        (time, lon, lat, alt) float64 1.03e+10 1.449e+10 ... 3.814e+10
    eTemperature  (time, lon, lat, alt) float64 180.0 179.9 ... 2.978e+03
    iTemperature  (time, lon, lat, alt) float64 180.0 181.4 ... 2.44e+03
    v_i_east      (time, lon, lat, alt) float64 -1.479 -0.9229 ... -183.3 -183.9
    v_i_north     (time, lon, lat, alt) float64 -5.483 -14.69 ... 51.56 51.49
    v_i_up        (time, lon, lat, alt) float64 -3.558 -5.341 ... -78.51 -99.6

# Now you can plot to your heart's content, blahblahblah
# Here we get down in the dirt with changing GITM.utility_programs.read_routines
---


In [2]:
gitm_dir = "/petastore/phil/GITM/SimStormForSal/GITM-outputs/"
dtime_storm_start = datetime.datetime(2011,5,21,13,40) 
gitm_file_pattern='3DALL*.bin'
cols=['all']
t_start_idx=4
t_end_idx=8
century_prefix='20'

flist = np.sort(glob.glob(os.path.join(gitm_dir, gitm_file_pattern)))

gitm_dtimes = []
for i in flist:
    yy, MM, dd = i[-17:-15], i[-15:-13], i[-13:-11]
    hr, mm, sec = i[-10:-8], i[-8:-6], i[-6:-4]
    try:
        gitm_dtimes.append(
            datetime.datetime(
                int(century_prefix + yy), int(MM), int(dd),
                int(hr), int(mm), int(sec)))
    except ValueError:
        raise ValueError(
            "GITM file name does not match expected format,",
            "filename %s cannot be parsed" % i)

if t_start_idx != 0:
    start_idx = gitm_dtimes.index(
        dtime_storm_start - datetime.timedelta(hours=t_start_idx))
    gitm_dtimes = gitm_dtimes[start_idx:]
    flist = flist[start_idx:]

if t_end_idx != -1:
    end_idx = gitm_dtimes.index(
        dtime_storm_start + datetime.timedelta(hours=t_end_idx))
    gitm_dtimes = gitm_dtimes[:end_idx]
    flist = flist[:end_idx]

f = read_routines.read_gitm_file(flist[0])

In [3]:
gitmgrid = {f["vars"][k].lower(): f[k][2:-2, 2:-2, 2:-2]
            for k in [0, 1, 2]}
nlons, nlats, nalts = np.array(f[0].shape) - 4  # ghost cells

In [11]:
f[0][2:-2, 2:-2, 2:-2].shape

(90, 180, 50)

In [32]:
gitmgrid['longitude'].shape

(90, 180, 50)

## Importing...
---

In [4]:
import datetime as dt
from netCDF4 import Dataset
from struct import unpack
import re

from aetherpy.utils.time_conversion import epoch_to_datetime
from aetherpy import logger


In [5]:
filename=flist[0]
file_vars=None
data = {"vars": []}
logger.info("Reading file : ", filename)

if not os.path.isfile(filename):
    raise IOError('input file does not exist')

with open(filename, 'rb') as fin:
    # Determine the correct endian
    end_char = '>'
    raw_rec_len = fin.read(4)
    rec_len = (unpack(end_char + 'l', raw_rec_len))[0]
    if rec_len > 10000 or rec_len < 0:
        # Ridiculous record length implies wrong endian.
        end_char = '<'
        rec_len = (unpack(end_char + 'l', raw_rec_len))[0]

    # Read version; read fortran footer+data.
    data["version"] = unpack(end_char + 'd', fin.read(rec_len))[0]

    _, rec_len = unpack(end_char + '2l', fin.read(8))

    # Read grid size information.
    data["nlons"], data["nlats"], data["nalts"] = unpack(
        end_char + 'lll', fin.read(rec_len))
    _, rec_len = unpack(end_char + '2l', fin.read(8))

    # Read number of variables.
    num_vars = unpack(end_char + 'l', fin.read(rec_len))[0]
    _, rec_len = unpack(end_char + '2l', fin.read(8))

    if file_vars is None:
        file_vars = np.arange(0, num_vars, 1)

    # Collect variable names in a list
    for ivar in range(num_vars):
        vcode = unpack(end_char + '%is' % (rec_len),
                        fin.read(rec_len))[0]
        var = vcode.decode('utf-8').replace(" ", "")
        data['vars'].append(var)
        dummy, rec_lec = unpack(end_char + '2l', fin.read(8))

    # Extract time
    rec_time = np.array(unpack(end_char + 'lllllll', fin.read(28)))
    rec_time[-1] *= 1000  # convert from millisec to microsec
    data["time"] = dt.datetime(*rec_time)

    # Header is this length:
    # Version + start/stop byte
    # nlons, nlats, nalts + start/stop byte
    # num_vars + start/stop byte
    # variable names + start/stop byte
    # time + start/stop byte

In [6]:
iheader_length = 84 + num_vars * 48

ntotal = data["nlons"] * data["nlats"] * data["nalts"]
idata_length = ntotal * 8 + 8

In [28]:
fin.seek(iheader_length + file_vars[0] * idata_length)

2004

In [13]:
{f['vars'][0]: f[0]}

{'Longitude': array([[[-0.10471976, -0.10471976, -0.10471976, ..., -0.10471976,
          -0.10471976, -0.10471976],
         [-0.10471976, -0.10471976, -0.10471976, ..., -0.10471976,
          -0.10471976, -0.10471976],
         [-0.10471976, -0.10471976, -0.10471976, ..., -0.10471976,
          -0.10471976, -0.10471976],
         ...,
         [-0.10471976, -0.10471976, -0.10471976, ..., -0.10471976,
          -0.10471976, -0.10471976],
         [-0.10471976, -0.10471976, -0.10471976, ..., -0.10471976,
          -0.10471976, -0.10471976],
         [-0.10471976, -0.10471976, -0.10471976, ..., -0.10471976,
          -0.10471976, -0.10471976]],
 
        [[-0.03490659, -0.03490659, -0.03490659, ..., -0.03490659,
          -0.03490659, -0.03490659],
         [-0.03490659, -0.03490659, -0.03490659, ..., -0.03490659,
          -0.03490659, -0.03490659],
         [-0.03490659, -0.03490659, -0.03490659, ..., -0.03490659,
          -0.03490659, -0.03490659],
         ...,
         [-0.0349065

In [7]:
fin = open(filename, 'rb')
sdata = unpack(end_char + 'l', fin.read(4))[0]
print(sdata)
whatisthis =  np.array(
                unpack(end_char + '%id' % (ntotal), fin.read(sdata))).reshape(
                    (data["nlons"], data["nlats"], data["nalts"]), order="F")
fin.close()

8


error: unpack requires a buffer of 7471872 bytes

(94, 184, 54)

In [24]:
gitmvars

NameError: name 'gitmvars' is not defined

In [25]:
xrgitmvars = ['Rho', 'O_3p_', 'O_2', 'N_2', 'N_4s_', 'NO', 'He', 'N_2d_', 'N_2p_', 'H', 'CO_2', 'O_1d_', 'Temperature', 'v_n_east', 'v_n_north', 'v_n_up', 'v_n_up_O_3p_', 'v_n_up_2n_', 'v_n_up_N_2', 'v_n_up_N_4s_', 'v_n_up_NO', 'v_n_up_He', 'O_4_spPlus_', 'NOPlus', 'O_2Plus', 'N_2Plus', 'NPlus', 'O_2dPlus', 'O_2pPlus', 'HPlus', 'HePlus', 'eMinus', 'eTemperature', 'iTemperature', 'v_i_east', 'v_i_north', 'v_i_up']
dimnames = ['time', 'lon', 'lat', 'alt']
testdict={}
for each in xrgitmvars:
    testdict[each] = (dimnames, [])

In [35]:
times=gitm_dtimes
lats, lons, alts  = np.unique(gitmgrid['latitude']), np.unique(gitmgrid['longitude']), np.unique(gitmgrid['altitude'])
testds = xr.Dataset(data_vars=testdict,
                    coords=dict(\
                        time=times,\
                        lon=lons,\
                        lat=lats,\
                        alt=alts)
)

ValueError: Could not convert tuple of form (dims, data[, attrs, encoding]): (['time', 'lon', 'lat', 'alt'], []) to Variable.

In [34]:
gitm_dtimes

[datetime.datetime(2011, 5, 21, 9, 40),
 datetime.datetime(2011, 5, 21, 9, 45),
 datetime.datetime(2011, 5, 21, 9, 50),
 datetime.datetime(2011, 5, 21, 9, 55),
 datetime.datetime(2011, 5, 21, 10, 0),
 datetime.datetime(2011, 5, 21, 10, 5),
 datetime.datetime(2011, 5, 21, 10, 10),
 datetime.datetime(2011, 5, 21, 10, 15),
 datetime.datetime(2011, 5, 21, 10, 20),
 datetime.datetime(2011, 5, 21, 10, 25),
 datetime.datetime(2011, 5, 21, 10, 30),
 datetime.datetime(2011, 5, 21, 10, 35),
 datetime.datetime(2011, 5, 21, 10, 40),
 datetime.datetime(2011, 5, 21, 10, 45),
 datetime.datetime(2011, 5, 21, 10, 50),
 datetime.datetime(2011, 5, 21, 10, 55),
 datetime.datetime(2011, 5, 21, 11, 0),
 datetime.datetime(2011, 5, 21, 11, 5),
 datetime.datetime(2011, 5, 21, 11, 10),
 datetime.datetime(2011, 5, 21, 11, 15),
 datetime.datetime(2011, 5, 21, 11, 20),
 datetime.datetime(2011, 5, 21, 11, 25),
 datetime.datetime(2011, 5, 21, 11, 30),
 datetime.datetime(2011, 5, 21, 11, 35),
 datetime.datetime(2011,